In [152]:
import pandas as pd
import os
import numpy as np
import keras
from keras import layers

# Confirm the files are in the correct location
assert os.path.exists('data/train.csv')
assert os.path.exists('data/test.csv')

# Load the training data csvs
train_csv = pd.read_csv('data/train.csv')
test_csv = pd.read_csv('data/test.csv')
test_csv.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [153]:
# Model / data parameters - used for reshaping
num_classes = 10
input_shape = (28, 28, 1)

# Convert the training table into x/y sets
train_ydata = pd.DataFrame(train_csv["label"], columns=["label"])
train_xdata = train_csv.iloc[:, 1:].to_numpy().reshape((train_csv.shape[0], *input_shape))

# Create x set for test set (for submission)
test_xdata = test_csv.to_numpy().reshape((test_csv.shape[0], *input_shape))

# One hot encoding for outputs
train_ydata = keras.utils.to_categorical(train_ydata, num_classes)


In [154]:
from sklearn.model_selection import train_test_split

# Normalize the pixel arrays 
train_xdata = train_xdata.astype(np.float32) / 255.0
test_xdata = test_xdata.astype(np.float32) / 255.0

# Split data to train and validation sets
x_train, x_val, y_train, y_val = train_test_split(train_xdata, train_ydata, test_size=0.1, random_state=0)


In [155]:
# Configure Model Structure
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model.summary()

Model: "sequential_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_40 (Conv2D)          (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d_40 (MaxPoolin  (None, 13, 13, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_41 (Conv2D)          (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_41 (MaxPoolin  (None, 5, 5, 64)         0         
 g2D)                                                            
                                                                 
 flatten_20 (Flatten)        (None, 1600)              0         
                                                                 
 dropout_20 (Dropout)        (None, 1600)            

In [156]:
# Train the model and evaluate
batch_size = 128
epochs = 100

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, 
validation_data=(x_val, y_val)
)

# score = model.evaluate(x_val, y_val, verbose=1)
# print("Validation loss:", score[0])
# print("Validation accuracy:", score[1])

Epoch 1/100
296/296 [==============================] - 12s 39ms/step - loss: 0.4338 - accuracy: 0.8697 - val_loss: 0.1262 - val_accuracy: 0.9624
Epoch 2/100
296/296 [==============================] - 10s 32ms/step - loss: 0.1234 - accuracy: 0.9621 - val_loss: 0.0863 - val_accuracy: 0.9740
Epoch 3/100
296/296 [==============================] - 9s 31ms/step - loss: 0.0925 - accuracy: 0.9718 - val_loss: 0.0693 - val_accuracy: 0.9762
Epoch 4/100
296/296 [==============================] - 9s 32ms/step - loss: 0.0745 - accuracy: 0.9762 - val_loss: 0.0596 - val_accuracy: 0.9814
Epoch 5/100
296/296 [==============================] - 9s 31ms/step - loss: 0.0665 - accuracy: 0.9799 - val_loss: 0.0507 - val_accuracy: 0.9838
Epoch 6/100
296/296 [==============================] - 9s 32ms/step - loss: 0.0584 - accuracy: 0.9821 - val_loss: 0.0475 - val_accuracy: 0.9843
Epoch 7/100
296/296 [==============================] - 9s 32ms/step - loss: 0.0527 - accuracy: 0.9835 - val_loss: 0.0411 - val_accurac

In [158]:
# Evaluate the model over the submission dataset
y_test = model.predict(test_xdata)

# Save the data to a file
output_data = pd.DataFrame({"ImageId": np.arange(len(y_test))+1, "Label": y_test.argmax(axis=-1)})
output_data.to_csv("prediction.csv", index=False)


875/875 [==============================] - 3s 4ms/step
